In [ ]:
import psycopg2
from collections import Counter
import pickle
import pandas as pd
import requests
import os
import re

# https://github.com/openeventdata/mordecai
#  $ python -m spacy download en_core_web_lg
#  $ docker pull elasticsearch:5.5.2
#  $ wget https://andrewhalterman.com/files/geonames_index.tar.gz
#  $ docker run -d -p 127.0.0.1:9200:9200 -v $(pwd)/geonames_index/:/usr/share/elasticsearch/data elasticsearch:5.5.2
from mordecai import Geoparser

# https://github.com/somnathrakshit/geograpy3
import geograpy

%load_ext nb_black

In [ ]:
def connect():
    conn = psycopg2.connect(
        user="postgres",
        password=os.environ.get("POSTGRES_PASS", ""),
        host="localhost",
        port=5432,
        database="venmo",
    )
    cur = conn.cursor()
    return conn, cur

In [ ]:
geo = Geoparser()

In [ ]:
mordecai_tokens = Counter()
geograpy_tokens = Counter()
meta = Counter()

conn, _ = connect()
with conn.cursor(name="exploration") as cursor:
    cursor.itersize = 2000
    cursor.execute("SELECT * FROM transactions")
    for i, row in enumerate(cursor):
        if i % 20000 == 0:
            print(i)
            with open('mordecai_tokens.pkl', 'wb') as f:
                pickle.dump(mordecai_tokens, f)
            with open('geograpy_tokens.pkl', 'wb') as f:
                pickle.dump(geograpy_tokens, f)
            with open('meta.pkl', 'wb') as f:
                pickle.dump(meta, f)
        msg = row[1]
        msg = re.sub(r"[^\w\d_\- ]", "", msg).strip()
        meta['msgs'] += 1
        if len(msg) == 0:
            continue
        meta['msgs_processed'] += 1
            
        geoplaces = geograpy.get_geoPlace_context(text=msg)
        for name, _ in geoplaces.city_mentions:
            geograpy_tokens[name] += 1
        
        try:
            g = geo.geoparse(msg)
            if len(g) == 0:
                continue
            for w in [item['word'] for item in g]:
                mordecai_tokens[w] += 1
        except:
            pass
            
conn.close()

In [ ]:
with open("mordecai_tokens.pkl", "rb") as f:
    mordecai_tokens = pickle.load(f)
with open("geograpy_tokens.pkl", "rb") as f:
    geograpy_tokens = pickle.load(f)
with open("meta.pkl", "rb") as f:
    meta = pickle.load(f)
meta

In [ ]:
mordecai_words.most_common(50)

In [ ]:
geograpy_tokens.most_common(50)